In [1]:
import os
import mido
from mido import MidiFile
import pretty_midi
import random

# from rich import print

In [36]:
raw_dir = os.path.join("..", "inputs", "all-fourbar-tmp")
files = os.listdir(raw_dir)
random.shuffle(files)

for filepath in files[:100]:
    if filepath.endswith("mid") or filepath.endswith("midi"):
        # midi_md = MidiFile(os.path.join(raw_dir, filepath))
        # for track in midi_md.tracks:
        #     for message in track:
        #         if message.type == "set_tempo":
        #             print(f"found tempo message in '{filepath}'", message)
        # midi_md.print_tracks()
        tempo = float(os.path.basename(filepath).split("-")[1])
        print(f"loop length should be {60 * 16 / tempo:.02f} s")
        midi = pretty_midi.PrettyMIDI(os.path.join(raw_dir, filepath))
        onsets = midi.get_onsets()
        print(f"{filepath}: {midi.get_end_time():.03f}s\n", onsets)

loop length should be 13.71 s
20240123-070-03_0946-0959_d02.mid: 13.713s
 [ 0.29448057  0.30227278  0.4074676   0.85551962  1.28409112  1.30357165
  1.30357165  1.73214315  1.76720809  2.01655878  2.01655878  2.14902633
  2.57759783  2.99058491  2.99058491  2.99837712  3.42694862  3.71136425
  3.71136425  3.86331233  4.29188383  4.73214365  4.73993585  4.73993585
  5.1217541   5.43733857  5.43733857  5.54253339  5.95552047  6.39967639
  6.41915691  6.41915691  6.85552062  7.10487131  7.10487131  7.25681939
  7.68539089  8.13344291  8.13344291  8.14123512  8.5503261   8.81915731
  8.83863784  8.96331318  9.4074691   9.82045618  9.84772891  9.85552112
 10.31136535 10.58019657 10.59578099 10.63084593 10.74772906 11.14902783
 11.56980712 11.57759933 11.61266427 11.99837862 12.26720984 12.27500205
 12.4074696  12.84772941 13.29188533 13.29188533]
loop length should be 13.71 s
20240123-070-03_1248-1261_u06.mid: 13.713s
 [ 0.01785715  0.31006499  0.31006499  0.31006499  0.7503248   0.75811701

In [20]:
raw_dir = os.path.join("..", "inputs", "all-fourbar-tmp")
files = os.listdir(raw_dir)[:10]
random.shuffle(files)
for filepath in files:
    if filepath.endswith("mid") or filepath.endswith("midi"):
        midi = MidiFile(os.path.join(raw_dir, filepath))
        pmdi = pretty_midi.PrettyMIDI(os.path.join(raw_dir, filepath))
        track_tempo = 300000
        for track in midi.tracks:
            for message in track:
                if message.type == "set_tempo":
                    print(f"got tempo {message.tempo}")
                    track_tempo = message.tempo
        print(
            f"{filepath} -> {midi.length:.03f} s -> {mido.tempo2bpm(track_tempo):.01f} bpm -> {midi.ticks_per_beat} tpb"
        )
        print(
            f"{filepath} -> {pmdi.get_end_time():.03f} s -> {pmdi.tick_to_time(midi.ticks_per_beat)} spb"
        )

got tempo 500000
20240124-064-02_0690-0705_d06.mid -> 8.002 s -> 120.0 bpm -> 220 tpb
20240124-064-02_0690-0705_d06.mid -> 8.000 s -> 0.5 spb
got tempo 500000
20240121-070-02_0973-0987_d05.mid -> 7.950 s -> 120.0 bpm -> 220 tpb
20240121-070-02_0973-0987_d05.mid -> 7.948 s -> 0.5 spb
got tempo 500000
20240122-055-05_0139-0157_d01.mid -> 8.002 s -> 120.0 bpm -> 220 tpb
20240122-055-05_0139-0157_d01.mid -> 8.000 s -> 0.5 spb
got tempo 500000
20240213-100-05_0297-0307_d02.mid -> 8.002 s -> 120.0 bpm -> 220 tpb
20240213-100-05_0297-0307_d02.mid -> 8.000 s -> 0.5 spb
20240124-064-02_0585-0600_n00.mid -> 8.000 s -> 200.0 bpm -> 220 tpb
20240124-064-02_0585-0600_n00.mid -> 8.000 s -> 0.5 spb
got tempo 500000
20231227-080-05_0000-0012_d02.mid -> 8.002 s -> 120.0 bpm -> 220 tpb
20231227-080-05_0000-0012_d02.mid -> 8.000 s -> 0.5 spb
got tempo 500000
20240121-070-03_0000-0013_d01.mid -> 8.002 s -> 120.0 bpm -> 220 tpb
20240121-070-03_0000-0013_d01.mid -> 8.000 s -> 0.5 spb
got tempo 500000
202401

In [3]:
def stretch_midi_file(file: MidiFile, new_duration_seconds):
    print(
        f"rescaling file from {file.length:.02f}s to {new_duration_seconds:.02f}s ({new_duration_seconds / file.length:.03f})"
    )
    # Calculate stretch factor based on the original duration
    stretch_factor = new_duration_seconds / file.length

    # Scale the time attribute of each message by the stretch factor
    for track in file.tracks:
        for msg in track:
            msg.time = int(msg.time * stretch_factor)

    return file

In [4]:
data_dir = os.path.join("..", "inputs", "all-time")
target_bpm = 100

file = os.listdir(data_dir)[0]
midi_file_path = os.path.join(data_dir, file)
midi_file = mido.MidiFile(midi_file_path)
bpm = float(os.path.basename(midi_file_path).split("-")[1])

print(f"found bpm {bpm} for file {os.path.basename(midi_file_path)}")
new_message = mido.MetaMessage("set_tempo", tempo=mido.bpm2tempo(target_bpm), time=0)
tempo_added = False
midi_file_cp = midi_file

midi_file.print_tracks()

found bpm 64.0 for file 20240124-064-02_0435-0442.mid
=== Track 0
MetaMessage('set_tempo', tempo=937500, time=0)
MetaMessage('set_tempo', tempo=937499, time=0)
MetaMessage('time_signature', numerator=4, denominator=4, clocks_per_click=24, notated_32nd_notes_per_beat=8, time=0)
MetaMessage('end_of_track', time=1)
=== Track 1
MetaMessage('track_name', name='20240124-064-02_0435-0442', time=0)
Message('program_change', channel=0, program=0, time=0)
Message('note_on', channel=0, note=45, velocity=18, time=5)
Message('note_on', channel=0, note=61, velocity=60, time=0)
Message('note_on', channel=0, note=54, velocity=24, time=112)
Message('note_on', channel=0, note=54, velocity=0, time=30)
Message('note_on', channel=0, note=61, velocity=0, time=144)
Message('note_on', channel=0, note=63, velocity=52, time=0)
Message('note_on', channel=0, note=54, velocity=17, time=44)
Message('note_on', channel=0, note=61, velocity=49, time=2)
Message('note_on', channel=0, note=63, velocity=0, time=9)
Message

In [5]:
with mido.open_output("Disklavier") as output:  # type: ignore
    for msg in midi_file.play(meta_messages=False):
        output.send(msg)

    print(len(midi_file_cp.tracks[0]))
    for track in midi_file_cp.tracks:
        # Remove existing set_tempo messages
        for j, msg in enumerate(track):
            if msg.type == "set_tempo":
                print(f"removing set tempo message {msg}")
                _ = track.pop(j)

        # Add new set_tempo message to the first track
        if not tempo_added:
            print(f"adding message {new_message}")
            track.insert(0, new_message)
            tempo_added = True

    new_time = midi_file.length * bpm / target_bpm
    midi_file_cp = stretch_midi_file(midi_file_cp, new_time)

    if os.path.exists("tmp.mid"):
        os.remove("tmp.mid")
    midi_file_cp.save("tmp.mid")
    midi_file_cp = MidiFile("tmp.mid")

    # confirm that changes stuck
    midi_file_cp.print_tracks()
    for msg in midi_file_cp.play(meta_messages=False):
        output.send(msg)

OSError: no ports available

In [41]:
probs = [
    1 / 20,  # same
    1 / 20,  # next 1
    1 / 20,  # prev 1
    1 / 20,  # next 2
    1 / 20,  # prev 2
    0.15,  # diff 1
    0.15,  # diff 2
    0.15,  # diff 3
    0.15,  # diff 4
    0.15,  # diff 5
]
probs.index(0.15)

5

In [6]:
tempos = range(50, 130, 10)
for t in tempos:
    print(f"{t} -> {mido.bpm2tempo(t)}")

50 -> 1200000
60 -> 1000000
70 -> 857143
80 -> 750000
90 -> 666667
100 -> 600000
110 -> 545455
120 -> 500000


In [4]:
midi = mido.MidiFile("../data/datasets/test/play/20231220-080-06_0029-0035.mid")
for track in midi.tracks:
    for msg in track:
        print(msg)

MetaMessage('set_tempo', tempo=750000, time=0)
MetaMessage('set_tempo', tempo=750000, time=0)
MetaMessage('time_signature', numerator=4, denominator=4, clocks_per_click=24, notated_32nd_notes_per_beat=8, time=0)
MetaMessage('end_of_track', time=1760)
MetaMessage('track_name', name='20231220-080-06_0029-0035', time=0)
program_change channel=0 program=0 time=0
note_on channel=0 note=33 velocity=62 time=220
note_on channel=0 note=65 velocity=73 time=2
note_on channel=0 note=62 velocity=67 time=3
note_on channel=0 note=65 velocity=0 time=61
note_on channel=0 note=62 velocity=0 time=7
note_on channel=0 note=57 velocity=65 time=32
note_on channel=0 note=57 velocity=0 time=55
note_on channel=0 note=57 velocity=70 time=58
note_on channel=0 note=57 velocity=0 time=66
note_on channel=0 note=60 velocity=83 time=142
note_on channel=0 note=64 velocity=79 time=0
note_on channel=0 note=64 velocity=0 time=76
note_on channel=0 note=60 velocity=0 time=14
note_on channel=0 note=38 velocity=51 time=23
not